# Quantifying Non-Shot Chances

This notebook uses the Logistic Regression Expected Goals model trained using StatsBomb event data on Metrica's sample tracking data (match 2) to calculate the expected goals and the non-shot expected goals of the match.

Starts with loading and preprocessing the Metrica tracking and event data by transforming the coordinates to be cinsistent with StatsBombs.

Traditional match stats are calculated from the event data.

Possessions are defined and identified using Metrica's possession definitions with event types 'BALL LOST' and 'BALL OUT'.

Expected goals and non-shot expected goals are calculated for both Home and Away teams across all frames of the match.

Expected goals are summed for each shot taken whilst non-shot expected goals include possessions which no shots were attempted. In these possessions, the maximum non-shot expected goal value is taken across the whole possession and attributed to that possession.

Outcomes are at the bottom. This allows possessions where shots weren't taken to still be accounted for in post match statistics, further analysis such as difference between expected goals and non-shot expected goals can be extrapolated upon and discussed. But having the ability to start the conversation is great.



In [1]:
import pandas as pd
import numpy as np


import Metrica_IO as mio
import Metrica_Functions_TLMAnalytics as mfun

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

## Load Metrica Tracking Data

Loading data using MetricaIO, need to convert to StatsBomb consistent coordinates since xG model was trained using that coordinate system. The coefficients of the Logistic Regressions are dependent on those coordinate distances and angles.

In [2]:
# Load in Metrica events and tracking data using Metrica_IO from Friends of Tracking
events = mio.read_event_data('sample-data-master/data',2)
tracking_home = mio.tracking_data('sample-data-master/data'
                                  , 2
                                  , 'Home')
tracking_away = mio.tracking_data('sample-data-master/data/'
                                  , 2
                                  , 'Away')
tracking_home.head(3)

Reading team: home
Reading team: away


,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,0.94275,0.50413,0.64812,0.28605,0.67752,0.42803,0.69882,0.55606,0.64348,0.76253,0.23339,0.25869,0.60269,0.47397,0.58508,0.53017,0.53163,0.75084,0.50637,0.30137,0.50194,0.61123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.08,0.94275,0.50413,0.64812,0.28605,0.67752,0.42803,0.69882,0.55606,0.64348,0.76253,0.23339,0.25869,0.60269,0.47397,0.58508,0.53017,0.53163,0.75084,0.50637,0.30137,0.50194,0.61123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.12,0.94275,0.50413,0.64812,0.28605,0.67752,0.42803,0.69882,0.55606,0.64348,0.76253,0.23339,0.25869,0.60269,0.47397,0.58508,0.53017,0.53163,0.75084,0.50637,0.30137,0.50194,0.61123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Convert Metrica coordinates to StatsBomb coordinates
events = mfun.to_statsbomb_coordinates(events)
tracking_home = mfun.to_statsbomb_coordinates(tracking_home)
tracking_away = mfun.to_statsbomb_coordinates(tracking_away)
tracking_home.tail(3)

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141154,2,5646.16,7.542,39.0056,46.1988,67.7152,27.0696,46.916,32.5704,54.2328,NaN,NaN,48.7104,73.5984,46.7832,34.9368,43.4592,35.0256,NaN,NaN,33.9864,51.1464,NaN,NaN,49.0044,68.4408,61.2348,4.2312,33.6372,50.6192,NaN,NaN
141155,2,5646.20,7.542,39.0056,46.1988,67.7152,27.0696,46.916,32.5704,54.2328,NaN,NaN,48.7104,73.5984,46.8012,35.0336,43.4592,35.0256,NaN,NaN,33.9192,51.1240,NaN,NaN,49.0044,68.4408,61.2336,4.1664,33.6708,50.6352,NaN,NaN
141156,2,5646.24,7.542,39.0056,46.1988,67.7152,27.0696,46.916,32.5704,54.2328,NaN,NaN,48.7104,73.5984,46.8084,35.0000,43.4592,35.0256,NaN,NaN,33.8412,51.1016,NaN,NaN,49.0044,68.4408,61.2348,4.1232,33.6780,50.8072,NaN,NaN


In [4]:
# reverse direction of play in the second half so that home team is always attacking from right->left
tracking_home,tracking_away,events = mfun.to_single_playing_direction(tracking_home,tracking_away,events)
tracking_home.tail(3)

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141154,2,5646.16,112.458,40.9944,73.8012,12.2848,92.9304,33.084,87.4296,25.7672,NaN,NaN,71.2896,6.4016,73.2168,45.0632,76.5408,44.9744,NaN,NaN,86.0136,28.8536,NaN,NaN,70.9956,11.5592,58.7652,75.7688,86.3628,29.3808,NaN,NaN
141155,2,5646.20,112.458,40.9944,73.8012,12.2848,92.9304,33.084,87.4296,25.7672,NaN,NaN,71.2896,6.4016,73.1988,44.9664,76.5408,44.9744,NaN,NaN,86.0808,28.8760,NaN,NaN,70.9956,11.5592,58.7664,75.8336,86.3292,29.3648,NaN,NaN
141156,2,5646.24,112.458,40.9944,73.8012,12.2848,92.9304,33.084,87.4296,25.7672,NaN,NaN,71.2896,6.4016,73.1916,45.0000,76.5408,44.9744,NaN,NaN,86.1588,28.8984,NaN,NaN,70.9956,11.5592,58.7652,75.8768,86.3220,29.1928,NaN,NaN


## Match Stats

Traditional stats such as goals, shots and passes are calculated here.

In [5]:
shots = events[events['Type'] == 'SHOT']
print('There are {} shots in total.'.format(shots.shape[0]))
home_shots = shots[shots['Team'] == 'Home']
print('There are {} shots by the Home team.'.format(home_shots.shape[0]))
home_goals = home_shots[home_shots['Subtype'].str.contains('GOAL')]
print('There are {} goals by the Home team.'.format(home_goals.shape[0]))
away_shots = shots[shots['Team'] == 'Away']
print('There are {} shots by the Away team.'.format(away_shots.shape[0]))
away_goals = away_shots[away_shots['Subtype'].str.contains('GOAL')]
print('There are {} goals by the Away team.'.format(away_goals.shape[0]))
passes = events[events['Type'] == 'PASS']
print('There are {} passes in total.'.format(passes.shape[0]))
home_passes = passes[passes['Team'] == 'Home']
print('There are {} passes by the Home team.'.format(home_passes.shape[0]))
away_passes = passes[passes['Team'] == 'Away']
print('There are {} passes by the Away team.'.format(away_passes.shape[0]))


There are 24 shots in total.
There are 13 shots by the Home team.
There are 3 goals by the Home team.
There are 11 shots by the Away team.
There are 2 goals by the Away team.
There are 964 passes in total.
There are 543 passes by the Home team.
There are 421 passes by the Away team.


## Define Possessions

Possessions change everytime a BALL OUT or BALL LOST occurs. Separate for Home / Away.

In [6]:
events['Type'].value_counts()

PASS              964
CHALLENGE         311
RECOVERY          248
BALL LOST         233
SET PIECE          80
BALL OUT           49
SHOT               24
FAULT RECEIVED     20
CARD                6
Name: Type, dtype: int64

Every time the event type is 'BALL LOST' or 'BALL OUT', the possession ends. The count increases to identify a new possession for the next event.

In [7]:
home_events = events[events['Team'] == 'Home']
home_events.loc[:, 'Type_shift'] = home_events.loc[:, 'Type'].shift(1)
possession_count = 1
for index, row in home_events.iterrows():
    if (home_events.loc[index, 'Type_shift'] == 'BALL LOST') | (home_events.loc[index, 'Type_shift'] == 'BALL OUT'):
        possession_count += 1
        home_events.loc[index, 'Possession'] = possession_count
    else:
        home_events.loc[index, 'Possession'] = possession_count
    possession_count = possession_count
home_events.drop(['Type_shift'], axis=1, inplace=True)
home_events.tail(10)

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y,Possession
1919,Home,CHALLENGE,GROUND-FAULT-WON,2,137429,5497.16,137429,5497.16,Player1,NaN,82.8,4.0,NaN,NaN,146.0
1920,Home,FAULT RECEIVED,NaN,2,137429,5497.16,137429,5497.16,Player1,NaN,82.8,4.0,NaN,NaN,146.0
1921,Home,SET PIECE,FREE KICK,2,138250,5530.00,138250,5530.00,Player3,NaN,NaN,NaN,NaN,NaN,146.0
1922,Home,BALL LOST,INTERCEPTION,2,138250,5530.00,138334,5533.36,Player3,NaN,80.4,4.8,34.8,42.4,146.0
1924,Home,CHALLENGE,AERIAL-FAULT-WON,2,138334,5533.36,138334,5533.36,Player7,NaN,36.0,40.0,NaN,NaN,147.0
1926,Home,SET PIECE,FREE KICK,2,139891,5595.64,139891,5595.64,Player9,NaN,NaN,NaN,NaN,NaN,147.0
1927,Home,SHOT,ON TARGET-SAVED,2,139891,5595.64,139925,5597.00,Player9,NaN,36.0,41.6,2.4,41.6,147.0
1931,Home,CHALLENGE,AERIAL-LOST,2,140180,5607.20,140180,5607.20,Player1,NaN,82.8,24.0,NaN,NaN,147.0
1933,Home,RECOVERY,NaN,2,140211,5608.44,140211,5608.44,Player12,NaN,79.2,25.6,NaN,NaN,147.0
1934,Home,BALL OUT,CLEARANCE,2,140265,5610.60,140298,5611.92,Player12,NaN,80.4,12.8,70.8,-3.2,147.0


Useful to record the frames which each possession starts and ends to merge onto tracking data later on.

In [8]:
home_possession_frames = home_events.groupby(['Possession']).agg({'Start Frame': 'min', 'End Frame':'max'}).reset_index()
print('There are {} possessions for the Home team.'.format(home_possession_frames.shape[0]))
home_possession_frames.head(5)

There are 147 possessions for the Home team.


,Possession,Start Frame,End Frame
0,1.0,504,534
1,2.0,1084,1121
2,3.0,1431,1611
3,4.0,1672,1724
4,5.0,1874,1918


Repeat for Away team.

In [9]:
away_events = events[events['Team'] == 'Away']
away_events.loc[:, 'Type_shift'] = away_events.loc[:, 'Type'].shift(1)
possession_count = 1
for index, row in away_events.iterrows():
    if (away_events.loc[index, 'Type_shift'] == 'BALL LOST') | (away_events.loc[index, 'Type_shift'] == 'BALL OUT'):
        possession_count += 1
        away_events.loc[index, 'Possession'] = possession_count
    else:
        away_events.loc[index, 'Possession'] = possession_count
    possession_count = possession_count
away_events.drop(['Type_shift'], axis=1, inplace=True)
away_possession_frames = away_events.groupby(['Possession']).agg({'Start Frame': 'min', 'End Frame':'max'}).reset_index()
print('There are {} possessions for the Away team.'.format(away_possession_frames.shape[0]))
away_events.tail(10)

There are 136 possessions for the Away team.


,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y,Possession
1902,Away,CHALLENGE,AERIAL-WON,2,136060,5442.40,136060,5442.40,Player 26,NaN,114.0,36.8,NaN,NaN,135.0
1903,Away,SHOT,HEAD-OFF TARGET-OUT,2,136060,5442.40,136078,5443.12,Player 26,NaN,114.0,36.8,123.6,32.8,135.0
1909,Away,CHALLENGE,GROUND-LOST,2,137026,5481.04,137026,5481.04,Player21,NaN,55.2,61.6,NaN,NaN,135.0
1918,Away,CHALLENGE,GROUND-FAULT-LOST,2,137416,5496.64,137416,5496.64,Player20,NaN,81.6,5.6,NaN,NaN,135.0
1923,Away,CHALLENGE,AERIAL-FAULT-LOST,2,138327,5533.08,138327,5533.08,Player17,NaN,34.8,38.4,NaN,NaN,135.0
1925,Away,CARD,YELLOW,2,138445,5537.80,138445,5537.80,Player17,NaN,NaN,NaN,NaN,NaN,135.0
1928,Away,RECOVERY,SAVED,2,139926,5597.04,139926,5597.04,Player25,NaN,1.2,41.6,NaN,NaN,135.0
1929,Away,PASS,NaN,2,140010,5600.40,140058,5602.32,Player25,Player21,15.6,40.0,43.2,64.0,135.0
1930,Away,BALL LOST,INTERCEPTION,2,140122,5604.88,140183,5607.32,Player21,NaN,55.2,61.6,85.2,19.2,135.0
1932,Away,CHALLENGE,AERIAL-LOST,2,140183,5607.32,140183,5607.32,Player24,NaN,81.6,24.0,NaN,NaN,136.0


## Calculate xG for all Frames

For each frame in the tracking data, create features for calculating the xG for each frame according to each specified attacking team.

Eg. for Home team, the distance calculated will be the distance to the opponents (playing_direction = left to right). And opposite for the Away team.

### Home

In [10]:
# Create distance, angle and defender features
home_tracking_frames = mfun.create_features(tracking_home = tracking_home,
                                       tracking_away = tracking_away,
                                       start_frame = 0,
                                       number_frames = tracking_home.shape[0],
                                       field_dimen = (120, 80),
                                       attacking_team = 'Home')
# Calculate xG using logistic regression model
home_tracking_frames = mfun.create_xG(home_tracking_frames)
# Calculate non-shot xG based on if the attacking team is near the ball
home_tracking_frames = mfun.create_non_shot_xG(home_tracking_frames, attacking_team = 'Home')
home_tracking_frames.head()

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG
0,1,0.04,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,1,0.08,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,1,0.12,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,1,0.16,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.9048,59.3040,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4032,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,1,0.20,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.8040,59.3032,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.3396,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


### Away

In [11]:
# Create distance, angle and defender features
away_tracking_frames = mfun.create_features(tracking_home = tracking_home,
                                           tracking_away = tracking_away,
                                           start_frame = 0,
                                           number_frames = tracking_home.shape[0],
                                           field_dimen = (120, 80),
                                           attacking_team = 'Away',
                                           playing_direction = 'left-right')
# Calculate xG using logistic regression model
away_tracking_frames = mfun.create_xG(away_tracking_frames)
# Calculate non-shot xG based on if the attacking team is near the ball
away_tracking_frames = mfun.create_non_shot_xG(away_tracking_frames, attacking_team = 'Away')
away_tracking_frames.head()

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG
0,1,0.04,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,1,0.08,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,1,0.12,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,1,0.16,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.9048,59.3040,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4032,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,1,0.20,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.8040,59.3032,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.3396,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


## Merge on Possession Indentifier by Frames

Respective Home / Away Possession columns identify the frames which that possession cover. So if Home Possession == 1 for a row, then it's the first possession for the Home team. Similar for Away team.

This is to ensure we only include the xG values for frames where the team has possession of the ball. We can also use the possessions to find the highest xG_available to the team during that possession.

### Home

In [12]:
# Map the Possession identifiers to the respective tracking frames
for index1, row1 in home_tracking_frames.iterrows():
    for index2, row2 in home_possession_frames.iterrows():
        if (index1 > home_possession_frames.loc[index2, 'Start Frame']) & (index1 < home_possession_frames.loc[index2, 'End Frame']):
            home_tracking_frames.loc[index1, 'Home Possession'] = home_possession_frames.loc[index2, 'Possession']
            next
home_tracking_frames.head()

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG,Home Possession
0,1,0.04,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,1,0.08,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,1,0.12,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,1,0.16,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.9048,59.3040,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4032,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,1,0.20,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.8040,59.3032,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.3396,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


### Away

In [13]:
# Map the Possession identifiers to the respective tracking frames
for index1, row1 in away_tracking_frames.iterrows():
    for index2, row2 in away_possession_frames.iterrows():
        if (index1 > away_possession_frames.loc[index2, 'Start Frame']) & (index1 < away_possession_frames.loc[index2, 'End Frame']):
            away_tracking_frames.loc[index1, 'Away Possession'] = away_possession_frames.loc[index2, 'Possession']
            next
away_tracking_frames.head()

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG,Away Possession
0,1,0.04,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,1,0.08,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,1,0.12,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,28.0068,59.3048,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4692,14.5232,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,1,0.16,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.9048,59.3040,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.4032,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,1,0.20,113.13,39.6696,77.7744,57.116,81.3024,45.7576,83.8584,35.5152,77.2176,18.9976,27.8040,59.3032,72.3228,42.0824,70.2096,37.5864,63.7956,19.9328,60.7644,55.8904,60.2328,31.1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4616,38.5896,39.3396,14.5224,36.8808,35.8544,38.682,48.8368,43.806,55.1608,59.4768,6.9088,47.7744,42.2256,51.9756,54.9168,59.8668,81.8104,60.2124,39.884,59.4204,51.0128,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


### Include a Shot Indicator

The shot indicator will allow the tracking data to know when an actual shot took place. For possessions where shots took place, the xG of the shot can be used to override and other non_shot_xG since the shot takes priority.

In [14]:
# Create shot indicator and merge to tracking data
home_shots['Shot Indicator'] = 1
home_tracking_frames = pd.merge(home_tracking_frames, home_shots[['Start Time [s]', 'Shot Indicator']], how='left', left_on='Time [s]', right_on='Start Time [s]')
home_tracking_frames.drop(['Start Time [s]'], axis=1, inplace=True)
home_tracking_frames[home_tracking_frames['Shot Indicator'] == 1]

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG,Home Possession,Shot Indicator
4418,1,176.76,89.2344,43.1432,12.7356,72.4264,41.0100,58.8976,48.4584,40.9808,29.9100,31.9096,15.5784,61.7808,28.9692,45.8976,10.8684,40.0832,15.0732,37.1216,5.1240,61.5696,6.8004,41.7656,NaN,NaN,NaN,NaN,NaN,NaN,15.2712,61.7568,1.2396,43.2752,6.7500,35.9824,5.9232,41.4272,5.3892,49.6744,6.8652,60.6816,11.5932,38.6544,9.8928,53.6088,11.0784,44.4784,18.1284,60.6552,37.4964,57.2328,32.2008,35.1632,NaN,NaN,26.581345,0.159782,3.062207,2.0,0.019134,0.308136,0.019134,7.0,1.0
12201,1,488.08,89.4348,42.1136,14.8872,60.0056,38.8380,58.7016,52.3488,39.1992,28.7604,28.3064,8.0796,48.0784,37.3908,44.5304,8.0688,48.1480,9.9564,37.8424,16.4064,17.3600,6.3096,35.6648,NaN,NaN,NaN,NaN,NaN,NaN,6.5580,36.5160,2.3316,40.3192,8.3508,33.7896,6.5988,35.7232,6.4284,46.4688,10.9176,57.9616,13.4808,34.9488,12.9936,48.7032,15.0960,40.7152,19.1748,51.6248,38.5152,58.7088,28.0848,29.6264,NaN,NaN,7.426010,0.854438,0.793849,1.0,0.330727,0.886704,0.330727,16.0,1.0
16483,1,659.36,101.1312,41.6400,66.6408,61.2456,65.7336,47.3912,74.8644,34.2560,75.7188,26.3896,27.5664,56.9176,58.0536,45.8032,62.2416,39.9184,55.2300,27.8584,17.5884,39.0392,12.9468,53.2672,NaN,NaN,NaN,NaN,NaN,NaN,13.0092,53.2904,4.1676,42.8440,14.7300,37.9832,11.2320,43.6728,11.4984,51.4880,36.6696,58.9584,63.9660,31.2696,20.4060,46.3936,35.5920,38.3232,73.2060,64.7120,77.6340,44.1040,75.4080,55.1432,NaN,NaN,18.597688,0.278761,2.351842,2.0,0.048740,0.066573,0.048740,21.0,1.0
27344,1,1093.80,87.1824,41.7352,18.1668,64.4352,39.3312,50.3256,10.2048,32.7296,57.2208,41.7944,44.1408,34.9896,17.1288,39.9992,8.9988,30.4144,14.9388,17.1008,26.0916,67.1160,8.3376,38.5528,NaN,NaN,NaN,NaN,NaN,NaN,10.3872,33.1424,3.8220,38.0144,24.4308,60.7608,9.9252,34.1216,8.0184,39.2304,13.2540,51.6048,14.8188,40.5552,18.6240,58.1008,12.8640,43.2800,28.3416,41.1200,21.6264,44.7488,11.0736,29.7216,NaN,NaN,12.446710,0.492677,1.082717,1.0,0.142581,0.451302,0.142581,33.0,1.0
29753,1,1190.16,92.1264,38.9536,19.3500,54.7016,49.6296,51.9928,55.6032,36.8200,24.5328,23.6600,15.0516,38.2616,42.5604,36.0304,13.9284,28.5000,12.5928,14.6424,28.0356,31.7720,10.1076,41.2696,NaN,NaN,NaN,NaN,NaN,NaN,24.6168,23.6496,3.0540,37.4104,12.6384,19.9912,13.5480,29.2376,12.0768,38.3864,11.8140,41.3768,18.0384,22.6400,20.9016,36.9336,18.8028,27.2640,22.4664,54.2024,47.0148,54.0720,48.4500,38.4328,NaN,NaN,29.552029,0.206422,6.655422,1.0,0.031972,0.084641,0.031972,35.0,1.0
56078,1,2243.16,86.8812,44.1416,13.6692,68.6040,42.4776,52.8176,50.6280,39.8784,34.0152,33.7248,12.5904,47.0608,35.8164,47.9080,12.4176,31.7128,20.8920,56.4576,28.3920,65.9024,11.3292,37.6912,NaN,NaN,NaN,NaN,NaN,NaN,12.6684,47.7416,2.1936,41.4696,9.7428,33.0792,9.6372,38.3760,8.3124,45.5016,11.7240,63.5288,15.8124,32.6072,18.2160,44.2304,14.0940,53.2208,20.8188,60.2800,40.1676,34.5504,30.3804,53.3064,NaN,NaN,14.846573,0.420855,4.898197,1.0,0.155117,0.685254,0.155117,65.0,1.0
67066,1,2682.68,91.7532,39.0696,23.9244,60.1184,53.9220,48.9056,53.6376,26.2224,32.9388,25.3240,23.7204,24.2776,42.2424,44.4088,15.6588,18.5792,20.3196,43.0264,14.0760,37.3648,9.5532,36.5432,NaN,NaN,NaN,NaN,NaN,NaN,15.6792,18.4888,4.0104,36.1312,13.9896,20.2760,9.3360,33.5616,10.3080,38.5128,9.6936,45.1272,14.6256,23.3424,18.3372,29.7848,17.7864,37.2848,20.6772,50.4344,50.5416,26.2856,23.3520,32.2592,NaN,NaN,26.618960,0.163513,2.459437,1.0,0.024509,0.09

In [15]:
# Create shot indicator and merge to tracking data
away_shots['Shot Indicator'] = 1
away_tracking_frames = pd.merge(away_tracking_frames, away_shots[['Start Time [s]', 'Shot Indicator']], how='left', left_on='Time [s]', right_on='Start Time [s]')
away_tracking_frames.drop(['Start Time [s]'], axis=1, inplace=True)
away_tracking_frames[away_tracking_frames['Shot Indicator'] == 1]

,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,Home_4_x,Home_4_y,Home_5_x,Home_5_y,Home_6_x,Home_6_y,Home_7_x,Home_7_y,Home_8_x,Home_8_y,Home_9_x,Home_9_y,Home_10_x,Home_10_y,Home_12_x,Home_12_y,Home_13_x,Home_13_y,Home_14_x,Home_14_y,ball_x,ball_y,Away_25_x,Away_25_y,Away_15_x,Away_15_y,Away_16_x,Away_16_y,Away_17_x,Away_17_y,Away_18_x,Away_18_y,Away_19_x,Away_19_y,Away_20_x,Away_20_y,Away_21_x,Away_21_y,Away_22_x,Away_22_y,Away_23_x,Away_23_y,Away_24_x,Away_24_y,Away_26_x,Away_26_y,distance,angle,distance_nearest_defender,number_blocking_defenders,xG,distance_nearest_attacker,non_shot_xG,Away Possession,Shot Indicator
18514,1,740.60,115.7292,41.3944,101.7096,51.4880,102.6804,45.6096,104.2860,34.5560,101.4156,18.2360,94.1424,44.6888,94.0284,40.5592,91.1124,29.8064,87.3444,7.3488,80.8764,51.5000,69.0252,31.6600,NaN,NaN,NaN,NaN,NaN,NaN,95.3916,52.0408,26.5776,40.1832,73.7928,13.0480,63.0900,30.1120,65.1696,37.2776,77.9616,57.6632,95.6904,20.1792,97.3188,41.2296,76.5504,35.1056,95.5584,52.0760,106.2144,38.1336,103.0992,49.0856,NaN,NaN,27.396245,0.239509,6.342138,0.0,0.055348,0.170474,0.055348,26.0,1.0
53048,1,2121.96,116.9964,39.2760,109.1556,42.4840,109.7688,31.2960,111.4464,30.9216,106.7136,21.7816,99.8724,30.6280,97.8324,41.4744,99.0972,22.0168,88.8564,58.6744,70.8384,29.1272,69.0648,40.7312,NaN,NaN,NaN,NaN,NaN,NaN,110.4768,43.5784,24.3168,42.4752,80.6364,13.6480,67.1796,27.2304,63.7668,44.9872,64.6680,61.3024,101.6724,21.8384,85.1256,34.1976,71.6448,45.7160,94.4760,52.7136,107.8152,30.7568,110.5464,43.5840,NaN,NaN,10.173312,0.658078,1.715599,0.0,0.293708,0.069825,0.293708,62.0,1.0
63361,1,2534.48,119.6184,37.8928,108.7056,49.2408,110.2896,58.3304,111.5364,41.9384,113.8632,34.4856,107.9868,40.0192,109.4280,45.0960,111.4728,40.4592,101.0352,58.7056,71.1408,24.7664,59.8560,40.1104,NaN,NaN,NaN,NaN,NaN,NaN,111.8136,40.6848,12.5148,43.7640,68.9568,22.0584,57.3408,30.7000,58.9320,41.1952,62.4420,55.6864,109.1964,31.5000,91.6428,37.8504,81.4476,47.8312,111.2712,63.6040,111.7860,40.5560,102.4452,43.8400,NaN,NaN,8.214992,0.836034,0.408705,1.0,0.295756,0.131724,0.295756,71.0,1.0
64771,1,2590.88,118.6536,39.5288,108.0276,50.3304,108.7104,41.7848,105.5148,38.0216,108.5532,37.5944,97.7232,43.4792,94.8216,37.7832,98.7408,33.5928,91.9584,51.7032,84.5772,52.1408,64.2132,35.7504,NaN,NaN,NaN,NaN,NaN,NaN,92.3496,35.6200,20.9700,41.0080,82.0140,16.4920,62.3688,29.9392,60.7644,42.1768,110.3040,39.4992,110.2524,34.3472,92.3712,35.6464,77.7204,35.6136,89.3280,49.4040,77.7000,23.0112,102.3516,65.0192,NaN,NaN,27.995161,0.256770,3.284847,0.0,0.037192,0.034110,0.037192,73.0,1.0
69886,2,2795.48,118.2540,41.0528,107.0988,61.2120,110.2344,52.8184,107.8584,39.8696,108.8448,35.5288,98.3520,64.0488,104.3760,53.7040,104.2800,46.5400,99.2052,45.7536,75.6612,56.5632,70.0488,39.9408,NaN,NaN,NaN,NaN,NaN,NaN,94.4976,52.8360,25.4388,40.2296,83.4084,24.3944,67.3500,39.6136,71.5368,56.0128,103.2228,68.9864,100.5588,26.1696,85.2900,42.3160,94.4988,52.7880,110.2020,66.6744,109.1784,48.0176,108.2184,34.7400,NaN,NaN,28.550574,0.228615,8.504228,1.0,0.045788,0.048015,0.045788,77.0,1.0
86190,2,3447.64,118.9272,41.1416,110.2620,44.1624,109.1400,34.3856,109.7580,36.4784,112.7628,38.5080,106.6056,46.0120,107.9280,41.0328,111.1560,47.7968,114.7332,64.3152,100.0380,39.2408,108.9084,56.8624,NaN,NaN,NaN,NaN,NaN,NaN,100.6596,45.8616,40.8108,40.6200,93.0864,28.1600,108.6024,31.7000,110.1012,34.5176,83.8056,52.5504,96.7020,43.5840,101.4264,66.4248,108.0024,51.4328,115.1136,71.4016,104.9304,43.2512,112.8504,40.5792,NaN,NaN,20.209142,0.343913,5.947902,0.0,0.125456,4.566187,NaN,90.0,1.0
90164,2,3606.60,117.8256,36.0600,97.8828,47.4856,114.2388,32.6272,106.7592,22.9080,109.8708,27.5520,110.4948,43.1968,109.3404,33.9176,102.6252,29.9360,73.8012,21.6016,103.5780,38.3576,55.3764,26.6400,NaN,NaN,NaN,NaN,NaN,NaN,112.8816,26.5600,17.4936,39.6232,71.8992,17.9728,59.6748,32.5728,56.8284,48.8296,76.0368,58.1536,112.9464,26.5808,99.5568,49.4936,76.7976,4

## Sum xG and Non Shot xG

Sum the shot expected goals and the non-shot expected goals using xG and non_shot_xG respectively for the possessions that the team has.

### Home

In [16]:
# Sum the xG for the shots
home_shot_xg = home_tracking_frames[home_tracking_frames['Shot Indicator'] == 1][['xG', 'Home Possession']]
print('Total xG for all shots: {}.'.format(home_shot_xg['xG'].sum()))
home_shot_xg

Total xG for all shots: 2.0422338503137234.


,xG,Home Possession
4418,0.019134,7.0
12201,0.330727,16.0
16483,0.048740,21.0
27344,0.142581,33.0
29753,0.031972,35.0
56078,0.155117,65.0
67066,0.024509,79.0
73982,0.467064,89.0
98879,0.335298,112.0
117217,0.113531,131.0


In [18]:
# Sum the non-shot xG for all possessions
home_non_shot_xG = home_tracking_frames.groupby('Home Possession').max()['non_shot_xG']
print('Total non-shot xG for all possessions: {}.'.format(home_non_shot_xG.sum()))
home_non_shot_xG

Total non-shot xG for all possessions: 6.471366602506773.


Home Possession
1.0      0.000173
2.0      0.000373
3.0      0.000130
4.0      0.000085
5.0      0.000184
           ...   
143.0         NaN
144.0    0.000018
145.0    0.000009
146.0    0.001678
147.0    0.014475
Name: non_shot_xG, Length: 146, dtype: float64

In [20]:
# If a possession had a shot, then attribute the xG to that possession
home_xG_combined = home_non_shot_xG.copy()
row_count=0
for possession in list(home_shot_xg['Home Possession']):
    for index in home_xG_combined.index:
        if possession == index:
            home_xG_combined[index] = home_shot_xg['xG'].iloc[row_count]
    row_count+=1
print('Combined xG and non-shot xG for all possessions: {}.'.format(home_xG_combined.sum()))
home_xG_combined

Combined xG and non-shot xG for all possessions: 5.3129587422610225.


Home Possession
1.0      0.000173
2.0      0.000373
3.0      0.000130
4.0      0.000085
5.0      0.000184
           ...   
143.0         NaN
144.0    0.000018
145.0    0.000009
146.0    0.001678
147.0    0.018879
Name: non_shot_xG, Length: 146, dtype: float64

### Away

In [22]:
away_shot_xg = away_tracking_frames[away_tracking_frames['Shot Indicator'] == 1][['xG', 'Away Possession']]
print('Total xG for all shots: {}.'.format(away_shot_xg['xG'].sum()))
away_shot_xg

Total xG for all shots: 2.0219916974382524.


,xG,Away Possession
18514,0.055348,26.0
53048,0.293708,62.0
63361,0.295756,71.0
64771,0.037192,73.0
69886,0.045788,77.0
86190,0.125456,90.0
90164,0.127539,91.0
111757,0.124502,120.0
115008,0.291622,120.0
132569,0.134575,130.0


In [23]:
away_non_shot_xG = away_tracking_frames.groupby('Away Possession').max()['non_shot_xG']
print('Total non-shot xG for all possessions: {}.'.format(away_non_shot_xG.sum()))
away_non_shot_xG[:10]

Total non-shot xG for all possessions: 3.8990711322285203.


Away Possession
1.0     0.002531
2.0     0.000991
3.0     0.002739
4.0     0.003080
5.0     0.007574
6.0     0.000847
7.0     0.000031
8.0     0.000782
9.0     0.000069
10.0    0.000391
Name: non_shot_xG, dtype: float64

In [24]:
away_xG_combined = away_non_shot_xG.copy()
row_count=0
for possession in list(away_shot_xg['Away Possession']):
    for index in away_xG_combined.index:
        if possession == index:
            away_xG_combined[index] = away_shot_xg['xG'].iloc[row_count]
    row_count+=1
print('Combined xG and non-shot xG for all possessions: {}.'.format(away_xG_combined.sum()))
away_xG_combined

Combined xG and non-shot xG for all possessions: 3.4645735037493606.


Away Possession
1.0      0.002531
2.0      0.000991
3.0      0.002739
4.0      0.003080
5.0      0.007574
           ...   
131.0    0.000900
132.0    0.016164
133.0    0.015819
134.0    0.006342
135.0    0.490506
Name: non_shot_xG, Length: 134, dtype: float64